<h2>导入包</h2>

In [56]:
import logging
from datetime import datetime
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from lightgbm import Dataset
import matplotlib.pyplot as plt


logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

<h2>读取数据</h2>

In [57]:
x_train = pd.read_csv("../data/A榜-训练集_分布式光伏发电预测_气象变量数据.csv", encoding="gbk")
y_train = pd.read_csv("../data/A榜-训练集_分布式光伏发电预测_实际功率数据.csv", encoding="gbk")
info_train = pd.read_csv("../data/A榜-训练集_分布式光伏发电预测_基本信息.csv", encoding="gbk")
x_test = pd.read_csv("../data/A榜-测试集_分布式光伏发电预测_气象变量数据.csv", encoding="gbk")
y_test = pd.read_csv("../data/submit_example.csv", encoding="utf-8")
info_test = pd.read_csv("../data/A榜-测试集_分布式光伏发电预测_基本信息.csv", encoding="gbk")

In [58]:
x_train = pd.merge(x_train, info_train[["光伏用户编号", "装机容量(kW)", "经度", "纬度"]], how="left", on="光伏用户编号")
x_train["时间"] = pd.to_datetime(x_train["时间"])
x_test = pd.merge(x_test, info_test[["光伏用户编号", "装机容量(kW)", "经度", "纬度"]], how="left", on="光伏用户编号")
x_test["时间"] = pd.to_datetime(x_test["时间"])

In [59]:
y_train = y_train.set_index(["光伏用户编号", "综合倍率", "时间"]).stack().reset_index().rename(columns={0:"target"})
y_train["level_3"] = y_train["level_3"].apply(lambda x: int(x[1:]))
y_train["时间"] = pd.to_datetime(y_train["时间"])
y_train["时间"] = y_train["时间"] + (y_train["level_3"] - 1) * 15 * pd.Timedelta(1, unit="minutes")
y_train = y_train.drop(columns=["level_3"])

y_test = y_test.set_index(["光伏用户编号", "综合倍率", "时间"]).stack().reset_index().rename(columns={0:"target"})
y_test["level_3"] = y_test["level_3"].apply(lambda x: int(x[1:]))
y_test["时间"] = pd.to_datetime(y_test["时间"])
y_test["时间"] = y_test["时间"] + (y_test["level_3"] - 1) * 15 * pd.Timedelta(1, unit="minutes")
y_test = y_test.drop(columns=["level_3"])

In [60]:
df_train = pd.merge(x_train, y_train, on=["光伏用户编号", "时间"], how="left")
df_test = pd.merge(x_test, y_test, on=["光伏用户编号", "时间"], how="left")
df = pd.concat([df_train, df_test], axis=0)

<h2>特征工程</h2>

<h3>时间特征</h3>

In [61]:
df["年"] = df["时间"].dt.year
df["季节"] = df["时间"].dt.quarter
df["月"] = df["时间"].dt.month
df["日"] = df["时间"].dt.day
df["周"] = df["时间"].dt.week
df["is_day"] = (df["时间"].dt.hour + 6) % 24 // 12
df["分"] = df["时间"].dt.minute // 15 + df["时间"].dt.hour * 4

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """


In [62]:
# df["分_"] = df["分"].copy()
# df = pd.get_dummies(df, columns=["分_"], prefix_sep="")

<h3>光伏用户编号</h3>

In [63]:
df["光伏用户编号_"] = df["光伏用户编号"].copy()
df = pd.get_dummies(df, columns=["光伏用户编号_"], prefix_sep="")

<h3>气象特征</h3>

In [64]:
df['100m风速（100m/s）'] = df['100m风速（100m/s）'] * np.sin(np.pi * df['100m风向（°)'] / 180)
# df['cos_100m风速（100m/s）'] = df['100m风速（100m/s）'] * np.cos(np.pi * df['100m风向（°)'] / 180)
df['10米风速（10m/s）'] = df['10米风速（10m/s）'] * np.sin(np.pi * df['10米风向（°)'] / 180)
# df['cos_10米风速（10m/s）'] = df['10米风速（10m/s）'] * np.cos(np.pi * df['10米风向（°)'] / 180)

<h3>历史值特征</h3>

In [65]:
dfs = []
for site, df_site in df.groupby("光伏用户编号"):
    df_site = df_site.sort_values("时间")
    df_site["辐照强度（J/m2） - 1"] = df_site["辐照强度（J/m2）"].shift(1) - df_site["辐照强度（J/m2）"]
    df_site["辐照强度（J/m2） - 8"] = df_site["辐照强度（J/m2）"].shift(8) - df_site["辐照强度（J/m2）"]
#     df_site["辐照强度（J/m2） - 2"] = df_site["辐照强度（J/m2）"].shift(2) - df_site["辐照强度（J/m2）"]
    dfs.append(df_site)
df = pd.concat(dfs, axis=0)

<h3>处理异常值</h3>

In [66]:
print(df_train["target"].nsmallest(3))
df_train[df_train["target"] < -8]
df.loc[207628:207633, ["光伏用户编号", "时间", "target"]]

207630   -8.8900
37316    -0.0085
39139    -0.0084
Name: target, dtype: float64


,光伏用户编号,时间,target
207628,f6,2022-08-15 20:30:00,-0.002
207629,f6,2022-08-15 20:45:00,-0.002
207630,f6,2022-08-15 21:00:00,-8.890
207631,f6,2022-08-15 21:15:00,-0.002
207632,f6,2022-08-15 21:30:00,NaN
207633,f6,2022-08-15 21:45:00,NaN


In [67]:
df.loc[207630, "target"] = -0.002

<h3>光照与当天最强光照的比值</h3>

In [68]:
df["日期"] = df["时间"].dt.date
day_max_values = df[["光伏用户编号", "日期", "辐照强度（J/m2）"]].groupby(by=["光伏用户编号", "日期"]).max()
day_max_values = day_max_values.rename(columns={x: x + "_max" for x in day_max_values.columns}).reset_index()
df = pd.merge(df, day_max_values, on=["光伏用户编号", "日期"], how="left").drop(columns=["日期"])
df["辐照强度（J/m2）_max"] = df["辐照强度（J/m2）"] / df["辐照强度（J/m2）_max"]

<h3>当天的平均光照</h3>

In [69]:
df["日期"] = df["时间"].dt.date
day_mean_values = df[["光伏用户编号", "日期", "辐照强度（J/m2）"]].groupby(by=["光伏用户编号", "日期"]).mean()
day_mean_values = day_mean_values.rename(columns={x: x + "_mean" for x in day_mean_values.columns}).reset_index()
df = pd.merge(df, day_mean_values, on=["光伏用户编号", "日期"], how="left").drop(columns=["日期"])

<h3>温度与当天最高温最低度的差值</h3>

In [70]:
df["日期"] = df["时间"].dt.date
day_max_values = df[["光伏用户编号", "日期", "温度（K）"]].groupby(by=["光伏用户编号", "日期"]).max()
day_min_values = df[["光伏用户编号", "日期", "温度（K）"]].groupby(by=["光伏用户编号", "日期"]).min()
day_max_values = day_max_values.rename(columns={x: x + "_max" for x in day_max_values.columns}).reset_index()
day_min_values = day_min_values.rename(columns={x: x + "_min" for x in day_min_values.columns}).reset_index()
df = pd.merge(df, day_max_values, on=["光伏用户编号", "日期"], how="left")
df = pd.merge(df, day_min_values, on=["光伏用户编号", "日期"], how="left").drop(columns=["日期"])
df["温度（K）_max"] = df["温度（K）_max"] - df["温度（K）"]
df["温度（K）_min"] = df["温度（K）"] - df["温度（K）_min"]

<h3>划分测试集</h3>

In [71]:
df_train = df[df["时间"] <= df_train["时间"].max()]
df_test = df[df["时间"] >= df_test["时间"].min()]

<h2>训练模型</h2>

<h3>评测指标</h3>

In [72]:
def score(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return 1 / (1 + rmse)

<h3>lightgbm模型</h3>

In [73]:
params_lgb = {
    "num_boost_round": 1000,
    'learning_rate': 0.02,
    'boosting_type': 'gbdt',
    'objective': 'mse',
    'metric': 'mse',
    'num_leaves': 127,
    'verbose': -1,
    'seed': 42,
    'n_jobs': -1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
    "early_stopping_round": 100
}

<h3>交叉验证</h3>

In [74]:
model_lgb = []
kfold = KFold(n_splits=5, random_state=42, shuffle=True)

x = df_train.drop(columns=["光伏用户编号", "时间"]).astype(np.float32)
y = x.pop("target")
mse = 0
for fold, (train_index, val_index) in enumerate(kfold.split(x, y)):
    logging.info(f'############ fold: {fold} ###########')
    x_train, x_val, y_train, y_val = x.iloc[train_index], x.iloc[val_index], y.iloc[train_index], y.iloc[val_index]
    trainset = Dataset(x_train, y_train)
    valset = Dataset(x_val, y_val)
    model = lgb.train(params_lgb, trainset, valid_sets=[trainset, valset], categorical_feature=["分"], callbacks=[lgb.log_evaluation(1000)])
    model.save_model("../models/lgb_%d.txt" % fold)
    model_lgb.append(model)
    val_pred = Series(model.predict(x_val, num_iteration=model.best_iteration), index=y_val.index).fillna(0)
    mse += mean_squared_error(y_val.fillna(0), val_pred)
rmse = np.sqrt(mse)
score = 1 / (1 + rmse)
logging.info(f"--------------本地分数 {score}--------------")

2024-03-22 17:15:12,025 : INFO : ############ fold: 0 ###########
c:\program files\python37\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\program files\python37\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['分']
  _log_warning('categorical_feature in Dataset is overridden.\n'
c:\program files\python37\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
c:\program files\python37\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[1000]	training's l2: 0.0136263	valid_1's l2: 0.0222014


2024-03-22 17:15:17,945 : INFO : ############ fold: 1 ###########
c:\program files\python37\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\program files\python37\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['分']
  _log_warning('categorical_feature in Dataset is overridden.\n'
c:\program files\python37\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
c:\program files\python37\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[1000]	training's l2: 0.0138553	valid_1's l2: 0.0221733


2024-03-22 17:15:31,842 : INFO : ############ fold: 2 ###########
c:\program files\python37\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\program files\python37\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['分']
  _log_warning('categorical_feature in Dataset is overridden.\n'
c:\program files\python37\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
c:\program files\python37\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


...

2024-03-22 17:15:49,968 : INFO : ############ fold: 3 ###########
c:\program files\python37\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\program files\python37\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['分']
  _log_warning('categorical_feature in Dataset is overridden.\n'
c:\program files\python37\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
c:\program files\python37\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[1000]	training's l2: 0.0134686	valid_1's l2: 0.0216983


2024-03-22 17:16:07,900 : INFO : ############ fold: 4 ###########
c:\program files\python37\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\program files\python37\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['分']
  _log_warning('categorical_feature in Dataset is overridden.\n'
c:\program files\python37\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
c:\program files\python37\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[1000]	training's l2: 0.0139035	valid_1's l2: 0.0220982


2024-03-22 17:16:25,970 : INFO : --------------本地分数 0.7512672859132133--------------


In [75]:
importance = DataFrame()
importance["特征"] = model.feature_name()
importance["重要性"] = 0
for model in model_lgb:
    importance["重要性"] = importance["重要性"] + model.feature_importance()
importance["重要性"] = importance["重要性"] / kfold.n_splits
importance.sort_values("重要性", ascending=False)[0:50]

,特征,重要性
33,辐照强度（J/m2）_mean,10994.2
0,气压(Pa）,10709.2
7,降水（m）,9773.2
1,相对湿度（%）,9498.8
17,日,8114.6
5,温度（K）,7034.6
35,温度（K）_min,6437.4
18,周,5897.8
2,云量,5478.4
6,辐照强度（J/m2）,5467.2


<h2>预测</h2>

In [76]:
x_test = df_test.drop(columns=["光伏用户编号", "时间"]).astype(np.float32)
y_test = x_test.pop("target")
y_pred = np.zeros((df_test.shape[0], ))
for model in model_lgb:
    y_pred += model.predict(x_test, num_iteration=model.best_iteration)
y_pred = y_pred / kfold.n_splits
df_test["target"] = y_pred

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [77]:
df_test = df_test[["光伏用户编号", "综合倍率", "年", "月", "日", "分", "target"]]
df_test["时间"] = df_test["年"].astype(str) + "-" + df_test["月"].astype(str) + "-" + df_test["日"].astype(str) + " 0:00"
df_test["分"] = "p" + (df_test["分"] + 1).astype(str)
df_test = df_test.drop(columns=["年", "月", "日"])

In [78]:
result = pd.pivot(df_test, index=["光伏用户编号", "综合倍率", "时间"], columns="分", values="target").reset_index()
result = result[result["综合倍率"].notnull()]
result["综合倍率"] = result["综合倍率"].astype(int)

In [79]:
result.to_csv("../data/%s.csv" % datetime.now().strftime("%Y%m%d_%H%M%S"), encoding="utf-8", index=False)